# Libraries

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

global original_window

# Selenium

In [2]:
# Initialize the Chrome WebDriver service
service = Service()

# Set Chrome options
options = webdriver.ChromeOptions()

# Start the WebDriver
driver = webdriver.Chrome(service=service, options=options)

In [3]:
# URL of the Intel CPU Data page
url = 'https://www.systemrequirementslab.com/cyri'

# Open the webpage
driver.get(url)

# Set an implicit wait time
driver.implicitly_wait(3)

# New Tab Function

In [4]:
def open_new_tab(driver, link):
    driver.execute_script("window.open(arguments[0].href, '_blank');", link)
    time.sleep(2)  # Wait for the new tab to load
    driver.switch_to.window(driver.window_handles[-1])

# Extract Game Requirements Function

In [5]:
def extract_game_requirements(driver):
    game_requeriments_list = []
    game_elements = driver.find_elements(By.TAG_NAME, "li")
    keywords = ("CPU:", "VIDEO CARD:", "RAM:")

    for game in game_elements:
        text = game.text.strip()
        if text.startswith(keywords):
            game_requeriments_list.append(text)

    # Ensure there are exactly 6 elements (3 for minimum, 3 for recommended)
    if len(game_requeriments_list) == 6:
        return {
            "minimun_cpu": game_requeriments_list[0],
            "minimun_gpu": game_requeriments_list[1],
            "minimun_ram": game_requeriments_list[2],
            "recommended_cpu": game_requeriments_list[3],
            "recommended_gpu": game_requeriments_list[4],
            "recommended_ram": game_requeriments_list[5],
        }
    else:
        return None  # Return None if the data isn't complete

# Extract All Game information Function

In [6]:
def process_game(driver, link):
    try:
        game_name = link.text.strip()  # Get the game name
        open_new_tab(driver, link)  # Open the game page in a new tab
        game_requirements = extract_game_requirements(driver)  # Extract requirements

        if game_requirements:
            game_requirements["name"] = game_name  # Add the game name to the result
            return game_requirements

    except Exception as e:
        print(f"Error processing game {game_name}: {e}")
        return None

    finally:
        driver.close()  # Close the new tab
        driver.switch_to.window(original_window)  # Switch back to the original tab

In [7]:
time.sleep(2)  # Wait for the page to load

original_window = driver.current_window_handle

# Collect all <a> links and select from index 50 to 70
links = driver.find_elements(By.TAG_NAME, "a")
all_game_data = []

if len(links) > 70:
    for i in range(50, 70):  # Iterate from index 50 to 69
        game_data = process_game(driver, links[i])
        if game_data:
            all_game_data.append(game_data)

In [8]:
# Create a DataFrame with the collected data
games_requirements_data = pd.DataFrame(all_game_data, columns=["name", "minimun_cpu", "minimun_gpu", "minimun_ram", "recommended_cpu", "recommended_gpu", "recommended_ram"])

driver.close()

In [9]:
games_requirements_data.to_csv('tables/games_requirement_data.csv', index=True)